<a href="https://colab.research.google.com/github/FanmeiWang/AIDI-1003/blob/main/Final_Project_Fanmei_Hongan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unidecode openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 7.3 MB/s eta 0:00:00


In [11]:
import os
import re
import glob
import html
import pandas as pd
from unidecode import unidecode

FOLDER_PATH = "/content/extracted_RS_raw_data"
FILE_PATTERN = "*.csv"
OUTPUT_CSV = "/content/cleaned_merged_data.csv"

TITLE_COL = "title"
SELFTEXT_COL = "selftext"
CREATED_UTC_COL = "created_utc"

#2. Text cleaning
def clean_text(text: str) -> str:
    """
    Performs basic text cleaning, including:
    - Removing [deleted], [removed], or empty values
    - Decoding HTML entities (e.g., &amp; -> &)
    - Using unidecode to handle strange unicode characters
    - Removing URLs
    - Keeping only letters, digits, basic punctuation
    - Lowercasing and removing extra whitespace
    """
    if not isinstance(text, str):
        return ""
    text_stripped = text.strip().lower()
    if text_stripped in ["[deleted]", "[removed]", ""]:
        return ""

    text = html.unescape(text)
    text = unidecode(text)
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"[^a-zA-Z0-9\s\.\,\!\?\']", " ", text)
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    return text

#3. Reading and merging multiple files
def read_and_merge(folder_path: str, file_pattern: str) -> pd.DataFrame:
    """
    Reads all files matching file_pattern from folder_path,
    merges them into a single DataFrame, and returns it.
    """
    files = sorted(glob.glob(os.path.join(folder_path, file_pattern)))
    if not files:
        print(f"[WARNING] No files found in: {folder_path}/{file_pattern}")
        return pd.DataFrame()

    all_dfs = []
    for file_path in files:
        print(f"[INFO] Reading: {file_path}")
        if file_path.endswith(".csv"):
            df = pd.read_csv(file_path, encoding="utf-8", keep_default_na=False)
        else:
            df = pd.read_excel(file_path, engine="openpyxl", keep_default_na=False)
        all_dfs.append(df)

    if not all_dfs:
        return pd.DataFrame()

    merged_df = pd.concat(all_dfs, ignore_index=True)
    print(f"[INFO] Successfully merged {len(files)} file(s). Total rows: {len(merged_df)}")
    return merged_df

#4. Main process: merging, cleaning, adding "month" column
df_merged = read_and_merge(FOLDER_PATH, FILE_PATTERN)

if df_merged.empty:
    print("[ERROR] No data to process. Exiting.")
else:

    if CREATED_UTC_COL in df_merged.columns:
        df_merged["created_dt"] = pd.to_datetime(df_merged[CREATED_UTC_COL], unit="s", errors="coerce")
        df_merged["month"] = df_merged["created_dt"].dt.strftime("%Y-%m")
    else:
        print(f"[WARNING] Column '{CREATED_UTC_COL}' not found. Unable to create 'month'.")
        df_merged["created_dt"] = None
        df_merged["month"] = None

    df_merged["combined_text"] = (
        df_merged.get(TITLE_COL, "").fillna("") + " " + df_merged.get(SELFTEXT_COL, "").fillna("")
    )

    df_merged["clean_text"] = df_merged["combined_text"].apply(clean_text)

    before_drop = len(df_merged)
    df_merged = df_merged[df_merged["clean_text"].str.strip() != ""]
    after_drop = len(df_merged)
    print(f"[INFO] Dropped {before_drop - after_drop} empty rows. Remaining rows: {after_drop}")

    output_path = os.path.join(FOLDER_PATH, OUTPUT_CSV)
    df_merged.to_csv(output_path, index=False, encoding="utf-8")
    print(f"[DONE] Cleaned data saved to: {output_path}")

    print("[INFO] DataFrame columns:", df_merged.columns.tolist())
    print("[INFO] Head of the DataFrame:")
    print(df_merged.head(3))


[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2022-12_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-01_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-02_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-03_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-04_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-05_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-06_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-07_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-08_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-09_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-10_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-11_filtered.csv
[INFO] Successfully merged 1

In [3]:
!pip install bertopic hdbscan umap-learn sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [10]:
# ============================================
# (1) Imports
# ============================================
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

# ============================================
# (2) Load cleaned data
# ============================================
DATA_CSV = "/content/cleaned_merged_data.csv"  # change this path if needed
df = pd.read_csv(DATA_CSV, encoding="utf-8")

print("[INFO] Columns:", df.columns.tolist())
print(df.head(3))

# Filter out empty rows
df = df[df["clean_text"].str.strip() != ""].reset_index(drop=True)
print("[INFO] Number of documents after filtering:", len(df))

# ============================================
# (3) Prepare documents for BERTopic
# ============================================
docs = df["clean_text"].tolist()

embedding_model = SentenceTransformer("all-mpnet-base-v2")
topic_model = BERTopic(
    embedding_model=embedding_model,
    verbose=True,
    calculate_probabilities=True
)

# Fit the model
topics, probs = topic_model.fit_transform(docs)
df["topic_id"] = topics

# ============================================
# (4) Basic topic info
# ============================================
topic_info = topic_model.get_topic_info()
print("[INFO] Topic info (head):")
print(topic_info.head(10))

# ============================================
# (5) Exclude outliers where topic_id = -1
# ============================================
df_no_outliers = df[df["topic_id"] != -1].copy()
print("[INFO] Removed outliers with topic_id = -1. Remaining docs:", len(df_no_outliers))

# ============================================
# (6) Month-topic distribution
# ============================================
if "month" in df_no_outliers.columns:
    # Group by month and topic_id
    month_topic_counts = (
        df_no_outliers.groupby(["month", "topic_id"]).size()
          .reset_index(name="count")
          .sort_values(["month", "count"], ascending=[True, False])
    )
    print("[INFO] Month-topic distribution (top rows):")
    print(month_topic_counts.head(10))
else:
    print("[WARNING] No 'month' column found in df_no_outliers; skipping month-based stats.")

# ============================================
# (7) Pivot to compare monthly differences
# ============================================
if "month" in df_no_outliers.columns:
    # Pivot
    pivot_data = month_topic_counts.pivot(
        index="month", columns="topic_id", values="count"
    ).fillna(0)

    # Sort by chronological order if month is like YYYY-MM
    pivot_data = pivot_data.sort_index()
    print("[INFO] Pivoted data shape:", pivot_data.shape)
    print(pivot_data.head())

    # Compute month-to-month difference
    diff_data = pivot_data.diff().fillna(0)

    # Flatten back to long format
    diff_long = diff_data.reset_index().melt(
        id_vars=["month"], var_name="topic_id", value_name="diff_value"
    )
    diff_long = diff_long.sort_values("diff_value", ascending=False)

    print("[INFO] Largest monthly jumps (head 10):")
    print(diff_long.head(10))

    # Pick the topic-month with the biggest jump
    if len(diff_long) > 0:
        most_changed = diff_long.iloc[0]
        max_topic = most_changed["topic_id"]
        max_month = most_changed["month"]
        max_diff = most_changed["diff_value"]

        print(f"[INFO] Topic {max_topic} in month {max_month} had the largest jump of {max_diff} docs.")

        # Extract the docs for that topic-month
        df_chosen = df_no_outliers[
            (df_no_outliers["topic_id"] == max_topic) &
            (df_no_outliers["month"] == max_month)
        ].copy()
        print("[INFO] Example docs for that topic & month:")
        print(df_chosen["clean_text"].head(5))

else:
    print("[WARNING] Skipping pivot/diff because no 'month' column found.")




[INFO] Columns: ['subreddit', 'title', 'selftext', 'created_utc', 'created_dt', 'month', 'combined_text', 'clean_text']
           subreddit                                              title  \
0  ImmigrationCanada  Received the nomination letter in EE but not r...   
1  ImmigrationCanada  PGWP approved, Can I apply for OHIP immediatel...   
2  ImmigrationCanada                 OHIP on Interim work authorization   

                                            selftext   created_utc  \
0                                          [deleted]  1.669855e+09   
1  I have the approval letter, just waiting for t...  1.669855e+09   
2                                          [removed]  1.669857e+09   

            created_dt    month  \
0  2022-12-01 00:39:40  2022-12   
1  2022-12-01 00:42:03  2022-12   
2  2022-12-01 01:17:41  2022-12   

                                       combined_text  \
0  Received the nomination letter in EE but not r...   
1  PGWP approved, Can I apply for OHIP immedi

2025-04-12 15:06:38,432 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/655 [00:00<?, ?it/s]

2025-04-12 15:09:13,715 - BERTopic - Embedding - Completed ✓
2025-04-12 15:09:13,717 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-12 15:09:32,699 - BERTopic - Dimensionality - Completed ✓
2025-04-12 15:09:32,701 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-12 15:10:33,686 - BERTopic - Cluster - Completed ✓
2025-04-12 15:10:33,696 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-12 15:10:34,849 - BERTopic - Representation - Completed ✓


[INFO] Topic info (head):
   Topic  Count                                    Name  \
0     -1   7807                        -1_to_the_and_my   
1      0    772       0_we_married_sponsorship_together   
2      1    695                   1_card_pr_travel_back   
3      2    502       2_experience_hours_company_worked   
4      3    256               3_lmia_offer_employer_job   
5      4    246           4_removed_deleted_help_please   
6      5    239              5_medical_exam_ime_upfront   
7      6    231          6_immigrate_move_canada_moving   
8      7    196  7_biometrics_biometric_fee_appointment   
9      8    177              8_wes_eca_degree_education   

                                      Representation  \
0  [to, the, and, my, for, in, permit, is, of, work]   
1  [we, married, sponsorship, together, her, comm...   
2  [card, pr, travel, back, ecopr, prtd, citizens...   
3  [experience, hours, company, worked, count, re...   
4  [lmia, offer, employer, job, exempt, poin